In [4]:
%matplotlib inline
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
    )

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download = True,
    transform = ToTensor()
    )

print(training_data)
print(test_data)

train_dataloader = DataLoader(dataset=training_data, batch_size=64)
test_dataloader = DataLoader(dataset=test_data, batch_size=64)

print(train_dataloader)
print(test_dataloader)

class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.liner_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
        nn.ReLU()
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.liner_relu_stack(x)
    return logits

model = NeuralNetwork()

print(model)

learing_rate = 1e-3
batch_size = 64
epochs = 100

loss_fn = nn.CrossEntropyLoss()
optimazer = torch.optim.SGD(model.parameters(), lr=learing_rate)

def train_loop(dataloader, model, loss_fn, optimazer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    pred = model(X)
    loss = loss_fn(pred, y)

    optimazer.zero_grad()
    loss.backward()
    optimazer.step()
    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)

      print(f"loss: {loss:>7f} [{current:>5d} / {size:>5d}]")

def test_loop(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= size
  correct /= size

  print(f"Test error: \n Accuracy {100*correct:>0.1f}% Avg loss {test_loss:>8f} \n")


torch.manual_seed(42)

for i in range(epochs):
  print(f"epoch {i+1} \n --------------------------")
  train_loop(dataloader = train_dataloader, model = model, loss_fn = loss_fn, optimazer = optimazer)
  test_loop(dataloader = test_dataloader, model = model, loss_fn = loss_fn)
print("Done!")


Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()
Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (liner_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)
epoch 1 
 --------------------------
loss: 2.304281 [    0 / 60000]
loss: 2.300185 [ 6400 / 60000]
loss: 2.295037 [12800 / 60000]
loss: 2.287416 [19200 / 60000]
loss: 2.284033 [25600 / 60000]
loss: 2.283469 [32000 / 60000]
loss: 2.257445 [38400 / 60000]
loss: 2.251000 [44800 / 60000]
loss: 2.253856 [51200 / 60000]
loss: 2.230295 [57600 / 60000]
Test error: 
 Accuracy 45.6% Avg loss 0.0351